# Preprocessing data

From this guide: https://scikit-learn.org/stable/modules/preprocessing.html

---
#### ⚠️ Any skipped sections are indicated with \[🚧\].
---

## Overview

The `sklearn.preprocessing` package provides several common utility functions and transformer classes to change raw feature vectors into a representation that is more suitable for the downstream estimators.

In general, learning algorithms **benefit from standardization of the data set**. If some **outliers** are present in the set, ***robust scalers or transformers*** are more appropriate. The behaviors of the different scalers, transformers, and normalizers on a dataset containing marginal outliers is highlighted in the example:
* 📚 [Compare the effect of different scalers on data with outliers](https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html#sphx-glr-auto-examples-preprocessing-plot-all-scaling-py).

## [Standardization, or mean removal and variance scaling](https://scikit-learn.org/stable/modules/preprocessing.html#standardization-or-mean-removal-and-variance-scaling)

**Standardization** of datasets is a **common requirement for many machine learning estimators** implemented in scikit-learn; they might behave badly if the individual features do not more or less look like standard normally distributed data: *Gaussian with zero mean and unit variance*.

In practice we often ignore the shape of the distribution and just transform the data to center it by removing the mean value of each feature, then scale it by dividing non-constant features by their standard deviation.

For instance, many elements used in the objective function of a learning algorithm (such as the `RBF` kernel of Support Vector Machines or the `l1` and `l2` regularizers of linear models) assume that all features are centered around zero and have variance in the same order. If a feature has a variance that is orders of magnitude larger than others, it might dominate the objective function and make the estimator unable to learn from other features correctly as expected.

The `preprocessing` module provides the `StandardScaler` utility class, which is a quick and easy way to perform the following operation on an array-like dataset:

In [1]:
from IPython.display import display
from sklearn import preprocessing
import numpy as np
X_train = np.array(
    [
        [ 1., -1.,  2.],
        [ 2.,  0.,  0.],
        [ 0.,  1., -1.]
    ]
)

scaler = preprocessing.StandardScaler().fit(X_train)
scaler

StandardScaler()

In [2]:
scaler.mean_

array([1.        , 0.        , 0.33333333])

In [3]:
scaler.scale_

array([0.81649658, 0.81649658, 1.24721913])

In [5]:
X_scaled = scaler.transform(X_train)
X_scaled

array([[ 0.        , -1.22474487,  1.33630621],
       [ 1.22474487,  0.        , -0.26726124],
       [-1.22474487,  1.22474487, -1.06904497]])

Scaled data has zero mean and unit variance:

In [6]:
X_scaled.mean(axis=0)

array([0., 0., 0.])

In [7]:
X_scaled.std(axis=0)

array([1., 1., 1.])

This class implements the `Transformer` API to compute the mean and standard deviation on a training set so as to be able to later re-apply the same transformation on the testing set. This class is hence suitable for use in the early steps of a `Pipeline`:

In [8]:
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [9]:
X, y = make_classification(random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
pipe = make_pipeline(StandardScaler(), LogisticRegression())
pipe.fit(X_train, y_train)  # apply scaling on training data

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

In [10]:
pipe.score(X_test, y_test)  # apply scaling on testing data, without leaking training data.

0.96

It is possible to disable either centering or scaling by either passing `with_mean=False` or `with_std=False` to the constructor of `StandardScaler`.

### Scaling features to a range

An alternative standardization is:
* scaling features to lie *between a given minimum and maximum value*, often between zero and one, or 
* so that the *maximum absolute value of each feature is scaled to unit size*. 

This can be achieved using `MinMaxScaler` or `MaxAbsScaler`, respectively.

The motivation to use this scaling include robustness to very small standard deviations of features and preserving zero entries in sparse data.

Here is an example to scale a toy data matrix to the `[0, 1]` range:

In [11]:
X_train = np.array(
    [
        [ 1., -1.,  2.],
        [ 2.,  0.,  0.],
        [ 0.,  1., -1.]
    ]
)

min_max_scaler = preprocessing.MinMaxScaler()
X_train_minmax = min_max_scaler.fit_transform(X_train)
X_train_minmax

array([[0.5       , 0.        , 1.        ],
       [1.        , 0.5       , 0.33333333],
       [0.        , 1.        , 0.        ]])

The same instance of the transformer can then be applied to some new test data unseen during the fit call: the same scaling and shifting operations will be applied to be consistent with the transformation performed on the train data:

In [12]:
X_test = np.array([[-3., -1.,  4.]])
X_test_minmax = min_max_scaler.transform(X_test)
X_test_minmax

array([[-1.5       ,  0.        ,  1.66666667]])

It is possible to introspect the scaler attributes to find about the exact nature of the transformation learned on the training data:

In [13]:
min_max_scaler.scale_

array([0.5       , 0.5       , 0.33333333])

In [14]:
min_max_scaler.min_

array([0.        , 0.5       , 0.33333333])

If `MinMaxScaler` is given an explicit `feature_range=(min, max)` the full formula is:
```python
X_std = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
X_scaled = X_std * (max - min) + min

```

⚠️ `MaxAbsScaler` works in a very similar fashion, but scales *in a way that the training data lies within the range `[-1, 1]`* by dividing through the largest maximum value in each feature. **It is meant for data that is already centered at zero or sparse data.**

Here is how to use the toy data from the previous example with this scaler:

In [15]:
X_train = np.array(
    [
        [ 1., -1.,  2.],
        [ 2.,  0.,  0.],
        [ 0.,  1., -1.]
    ]
)

max_abs_scaler = preprocessing.MaxAbsScaler()

X_train_maxabs = max_abs_scaler.fit_transform(X_train)
X_train_maxabs

array([[ 0.5, -1. ,  1. ],
       [ 1. ,  0. ,  0. ],
       [ 0. ,  1. , -0.5]])

In [16]:
X_test = np.array([[ -3., -1.,  4.]])
X_test_maxabs = max_abs_scaler.transform(X_test)
X_test_maxabs

array([[-1.5, -1. ,  2. ]])

In [17]:
max_abs_scaler.scale_

array([2., 1., 2.])

### Scaling sparse data

Centering sparse data would destroy the sparseness structure in the data, and thus rarely is a sensible thing to do. However, it can make sense to scale sparse inputs, especially if features are on different scales.

`MaxAbsScaler` was **specifically designed for scaling sparse data**, and is the recommended way to go about this. However, `StandardScaler` can accept `scipy.sparse` matrices as input, *as long as `with_mean=False` is explicitly passed to the constructor*. Otherwise a `ValueError` will be raised as silently centering would break the sparsity and would often crash the execution by allocating excessive amounts of memory unintentionally. `RobustScaler` **cannot** be fitted to sparse inputs, but you can use the transform method on sparse inputs.

Note that the scalers accept both `Compressed Sparse Rows` and `Compressed Sparse Columns` format (see `scipy.sparse.csr_matrix` and `scipy.sparse.csc_matrix`). Any other sparse input will be converted to the `Compressed Sparse Rows` representation. To avoid unnecessary memory copies, it is recommended to choose the `CSR` or `CSC` representation *upstream*.

Finally, if the centered data is expected to be small enough, explicitly converting the input to an array using the `toarray` method of sparse matrices is another option.

### Scaling data with outliers

If your data contains **many outliers**, scaling using the `mean` and `variance` of the data **is likely to not work very well**. In these cases, you can use `RobustScaler` as a drop-in replacement instead. 
> It uses more robust estimates for the center and range of your data.

---

**🔍 Scaling vs Whitening**

It is sometimes not enough to center and scale the features independently, since a downstream *model can further make some assumption on the linear independence of the features*.

To address this issue you can use `PCA` with `whiten=True` to **further remove the linear correlation** across features.

---

### Centering kernel matrices

If you have a kernel matrix of a kernel $K$ that computes a dot product in a feature space $\phi$ defined by function $\phi$, a `KernelCenterer` can transform the kernel matrix so that it contains inner products in the feature space defined by $\phi$ followed by removal of the mean in that space.

## [Non-linear transformation](https://scikit-learn.org/stable/modules/preprocessing.html#non-linear-transformation)

Two types of transformations are available: **quantile transforms** and **power transforms**. Both quantile and power transforms are based on monotonic transformations of the features and thus preserve the rank of the values along each feature.

**Quantile transforms** put all features into the same desired distribution based on the formula $G^{-1}(F(X))$ where:
* $F$ is the *cumulative distribution function* of the feature and
* $G^{-1}$ is the *quantile function* of the desired output distribution $G$.

This formula is using the two following facts: 
* (i) if $X$ is a random variable with a continuous cumulative distribution function $F$ then $F(X)$ is uniformly distributed on `[0, 1]`; 
* (ii) if $U$ is a random variable with uniform distribution on $[0, 1]$ then $G^{-1}(U)$ has distribution $G$. 

ℹ️ By performing a rank transformation, a quantile transform smooths out unusual distributions and is **less influenced by outliers** than scaling methods.
⚠️ It does, however, **distort correlations and distances** within and across features.

**Power transforms** are a family of parametric transformations that aim to map data from any distribution to as close to a Gaussian distribution.

###  Mapping to a Uniform distribution

QuantileTransformer provides a non-parametric transformation to map the data to a uniform distribution with values between 0 and 1:

In [18]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [23]:
X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

print("X_train[:3]:\n", X_train[:3])

X_train[:3]:
 [[5.9 3.  4.2 1.5]
 [5.8 2.6 4.  1.2]
 [6.8 3.  5.5 2.1]]


In [24]:
quantile_transformer = preprocessing.QuantileTransformer(random_state=0)  # This.
X_train_trans = quantile_transformer.fit_transform(X_train)
X_test_trans = quantile_transformer.transform(X_test)

/mnt/space/miniconda3/envs/py38_playaround-sk/lib/python3.8/site-packages/sklearn/preprocessing/_data.py:2612: UserWarning: n_quantiles (1000) is greater than the total number of samples (112). n_quantiles is set to n_samples.
  warnings.warn("n_quantiles (%s) is greater than the total number "


In [25]:
np.percentile(X_train[:, 0], [0, 25, 50, 75, 100]) 

array([4.3, 5.1, 5.8, 6.5, 7.9])

This feature corresponds to the sepal length in cm. Once the quantile transformation applied, those landmarks approach closely the percentiles previously defined:

In [28]:
np.percentile(X_train_trans[:, 0], [0, 25, 50, 75, 100]) 

array([0.        , 0.23873874, 0.50900901, 0.74324324, 1.        ])

This can be confirmed on a independent testing set with similar remarks:

In [29]:
np.percentile(X_test[:, 0], [0, 25, 50, 75, 100])

array([4.4  , 5.125, 5.75 , 6.175, 7.3  ])

In [30]:
np.percentile(X_test_trans[:, 0], [0, 25, 50, 75, 100])

array([0.01351351, 0.25      , 0.47747748, 0.60472973, 0.94144144])

### Mapping to a Gaussian distribution

🚧

## [Normalization](https://scikit-learn.org/stable/modules/preprocessing.html#normalization)

> **Normalization** is the process of **scaling individual samples to have unit norm**. 

This process can be useful if you *plan to use a quadratic form such as the dot-product or any other kernel to quantify the **similarity of any pair of samples***.

This assumption is the base of the [Vector Space Model](https://en.wikipedia.org/wiki/Vector_Space_Model) often used in text classification and clustering contexts.

The function `normalize` provides a quick and easy way to perform this operation on a single array-like dataset, either using the `l1`, `l2`, or `max` norms:

In [32]:
X = [[ 1., -1.,  2.],
     [ 2.,  0.,  0.],
     [ 0.,  1., -1.]]
X_normalized = preprocessing.normalize(X, norm='l2')

X_normalized

array([[ 0.40824829, -0.40824829,  0.81649658],
       [ 1.        ,  0.        ,  0.        ],
       [ 0.        ,  0.70710678, -0.70710678]])

💡 The `preprocessing` module further provides a utility class `Normalizer` that implements the same operation using the `Transformer` API **(even though the fit method is useless in this case: the class is stateless as this operation treats samples independently)**.

This class is hence suitable for use in the early steps of a `Pipeline`:

In [33]:
normalizer = preprocessing.Normalizer().fit(X)  # fit does nothing
normalizer

Normalizer()

The normalizer instance can then be used on sample vectors as any transformer:

In [34]:
normalizer.transform(X)

array([[ 0.40824829, -0.40824829,  0.81649658],
       [ 1.        ,  0.        ,  0.        ],
       [ 0.        ,  0.70710678, -0.70710678]])

In [36]:
normalizer.transform([[-1.,  1., 0.]])

array([[-0.70710678,  0.70710678,  0.        ]])

**Note:** L2 normalization is also known as *spatial sign preprocessing*.

---

**🔍 Sparse input**

`normalize` and` Normalizer` **accept** both dense array-like and sparse matrices from `scipy.sparse` as input.

For sparse input the data is **converted to the `Compressed Sparse Rows` representation** (see `scipy.sparse.csr_matrix`) before being fed to efficient Cython routines. To avoid unnecessary memory copies, it is recommended to choose the CSR representation upstream.

---

## [Encoding categorical features](https://scikit-learn.org/stable/modules/preprocessing.html#encoding-categorical-features)

Often features are not given as continuous values but categorical. For example a person could have features: 
* `["male", "female"]`, 
* `["from Europe", "from US", "from Asia"]`, 
* `["uses Firefox", "uses Chrome", "uses Safari", "uses Internet Explorer 🤣"]`. 

Such features can be efficiently coded as integers, for instance: 
* `["male", "from US", "uses Internet Explorer 🤣"]` could be expressed as `[0, 1, 3]` while 
* `["female", "from Asia", "uses Chrome"]` would be `[1, 2, 1]`.

To convert categorical features to such integer codes, we can use the `OrdinalEncoder`. This estimator transforms each categorical feature to one new feature of integers (`0` to `n_categories - 1`):

In [37]:
enc = preprocessing.OrdinalEncoder()
X = [
    ['male', 'from US', 'uses Safari'], 
    ['female', 'from Europe', 'uses Firefox']
]
enc.fit(X)

OrdinalEncoder()

In [38]:
enc.transform([['female', 'from US', 'uses Safari']])

array([[0., 1., 1.]])

‼️ Such integer representation can, however, **not** be used directly with all scikit-learn estimators, as these expect continuous input, and would interpret the categories as being ordered, which is often not desired (i.e. the set of browsers was ordered arbitrarily).

Another possibility to convert categorical features to features that can be used with scikit-learn estimators is to use a *one-of-K*, also known as *one-hot* or *dummy* encoding. This type of encoding can be obtained with the `OneHotEncoder`, which transforms each categorical feature with `n_categories` possible values into `n_categories` *binary features*, with one of them 1, and all others 0.

In [39]:
enc = preprocessing.OneHotEncoder()
X = [
    ['male', 'from US', 'uses Safari'], 
    ['female', 'from Europe', 'uses Firefox']
]
enc.fit(X)

OneHotEncoder()

In [40]:
enc.transform(
    [
        ['female', 'from US', 'uses Safari'],
        ['male', 'from Europe', 'uses Safari']
    ]
).toarray()

array([[1., 0., 0., 1., 0., 1.],
       [0., 1., 1., 0., 0., 1.]])

By default, the values each feature can take is inferred automatically from the dataset and can be found in the `categories_ attribute`:

In [41]:
enc.categories_

[array(['female', 'male'], dtype=object),
 array(['from Europe', 'from US'], dtype=object),
 array(['uses Firefox', 'uses Safari'], dtype=object)]

It is **possible to specify this explicitly** using the parameter categories. 

There are two genders, four possible continents and four web browsers in our dataset:

In [42]:
genders = ['female', 'male']
locations = ['from Africa', 'from Asia', 'from Europe', 'from US']
browsers = ['uses Chrome', 'uses Firefox', 'uses IE', 'uses Safari']
enc = preprocessing.OneHotEncoder(
    categories=[genders, locations, browsers]  # <-- See this.
)

In [43]:
# Note that there are missing categorical values for the 2nd and 3rd feature.

X = [
    ['male', 'from US', 'uses Safari'], 
    ['female', 'from Europe', 'uses Firefox']
]
enc.fit(X)

OneHotEncoder(categories=[['female', 'male'],
                          ['from Africa', 'from Asia', 'from Europe',
                           'from US'],
                          ['uses Chrome', 'uses Firefox', 'uses IE',
                           'uses Safari']])

In [44]:
enc.transform([['female', 'from Asia', 'uses Chrome']]).toarray()

array([[1., 0., 0., 1., 0., 0., 1., 0., 0., 0.]])

‼️ If there is a possibility that the training data might have missing categorical features, it can often be better to specify `handle_unknown='ignore'` instead of setting the categories manually as above. 

When `handle_unknown='ignore'` is specified and unknown categories are encountered during transform, no error will be raised but the resulting one-hot encoded columns for this feature will be all zeros. 

(`handle_unknown='ignore'` is only supported for one-hot encoding):

In [45]:
enc = preprocessing.OneHotEncoder(
    handle_unknown='ignore'  # <-- This.
)

X = [['male', 'from US', 'uses Safari'], ['female', 'from Europe', 'uses Firefox']]
enc.fit(X)

OneHotEncoder(handle_unknown='ignore')

In [46]:
enc.transform([['female', 'from Asia', 'uses Chrome']]).toarray()

array([[1., 0., 0., 0., 0., 0.]])

It is also possible to encode each column into `n_categories - 1` columns instead of `n_categories` columns by using the **`drop` parameter**.

This parameter allows the user to specify a category for each feature to be dropped. This is useful to avoid co-linearity in the input matrix in some classifiers. 

Such functionality is useful, for example, when using non-regularized regression (`LinearRegression`), since co-linearity would cause the covariance matrix to be non-invertible. 

When this parameter is not `None`, `handle_unknown` must be set to `error`.

In [47]:
X = [
    ['male', 'from US', 'uses Safari'],
    ['female', 'from Europe', 'uses Firefox']
]

drop_enc = preprocessing.OneHotEncoder(
    drop='first'  # Options: {‘first’, ‘if_binary’} or a array-like of shape (n_features,), default=None
).fit(X)
drop_enc.categories_

[array(['female', 'male'], dtype=object),
 array(['from Europe', 'from US'], dtype=object),
 array(['uses Firefox', 'uses Safari'], dtype=object)]

In [48]:
drop_enc.transform(X).toarray()
# Note that there are now FEWER features (3 not 6), as for each of the 2-category feature, we get only 1 column.

array([[1., 1., 1.],
       [0., 0., 0.]])

One might want to drop one of the two columns only for features with 2 categories. In this case, you can set the parameter `drop='if_binary'`.

In [49]:
X = [['male', 'US', 'Safari'],
     ['female', 'Europe', 'Firefox'],
     ['female', 'Asia', 'Chrome']]
drop_enc = preprocessing.OneHotEncoder(drop='if_binary').fit(X)
print(drop_enc.categories_)
display(drop_enc.transform(X).toarray())

[array(['female', 'male'], dtype=object), array(['Asia', 'Europe', 'US'], dtype=object), array(['Chrome', 'Firefox', 'Safari'], dtype=object)]


array([[1., 0., 0., 1., 0., 0., 1.],
       [0., 0., 1., 0., 0., 1., 0.],
       [0., 1., 0., 0., 1., 0., 0.]])

In the transformed `X` above, the first column is the encoding of the feature with categories `"male"`/`"female"`, while the remaining 6 columns is the encoding of the 2 features with respectively 3 categories each.

`OneHotEncoder` supports categorical features **with missing values** by considering the missing values **as an additional category**:

In [52]:
X = [
    ['male', 'Safari'],
    ['female', None],    # Browser missing.
    [np.nan, 'Firefox']  # Gender missing.
]
enc = preprocessing.OneHotEncoder(handle_unknown='error').fit(X)
print(enc.categories_)
print("^ Note the missing items as a separate category.", end="\n\n")
display(enc.transform(X).toarray())

[array(['female', 'male', nan], dtype=object), array(['Firefox', 'Safari', None], dtype=object)]
^ Note the missing items as a separate category.



array([[0., 1., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 1.],
       [0., 0., 1., 1., 0., 0.]])

⚠️ If a feature contains both `np.nan` and `None`, **they will be considered separate categories**:

In [54]:
X = [
    ['Safari'], 
    [None], 
    [np.nan], 
    ['Firefox']
]
enc = preprocessing.OneHotEncoder(handle_unknown='error').fit(X)
print(enc.categories_)
print("^ Note BOTH NaN and None!.", end="\n\n")
display(enc.transform(X).toarray())

[array(['Firefox', 'Safari', None, nan], dtype=object)]
^ Note BOTH NaN and None!.



array([[0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.]])

## [Discretization](https://scikit-learn.org/stable/modules/preprocessing.html#discretization)

[Discretization](https://en.wikipedia.org/wiki/Discretization_of_continuous_features) (otherwise known as *quantization* or *binning*) provides a way to partition continuous features into discrete values.

Certain datasets with continuous features may benefit from discretization, because discretization can transform the dataset of continuous attributes to one with only nominal attributes.

One-hot encoded discretized features can make a model more expressive, while maintaining interpretability. For instance, pre-processing with a discretizer can introduce nonlinearity to linear models.

### K-bins discretization

`KBinsDiscretizer` discretizes features into `k` bins:

In [55]:
X = np.array(
    [
        [ -3., 5., 15 ],
        [  0., 6., 14 ],
        [  6., 3., 11 ]
    ]
)

# Fit:
est = preprocessing.KBinsDiscretizer(n_bins=[3, 2, 2], encode='ordinal').fit(X)

⚠️ By default the output is one-hot encoded into **a sparse matrix** (See [Encoding categorical features](https://scikit-learn.org/stable/modules/preprocessing.html#preprocessing-categorical-features)) and this can be configured with the `encode` parameter.

For each feature, the bin edges are computed during `fit` and together with the number of bins, they will define the intervals. Therefore, for the current example, these intervals are defined as:
* feature 1: $[-\infty, -1), [-1,2), [2, \infty)$
* feature 2: $[-\infty, 5), [5, \infty)$
* feature 3: $[-\infty, 14), [14, \infty)$

In [66]:
print(est.bin_edges_)
print(est.n_bins_)
# 🤔 Hmm, not sure the above ranges are strictly true?

[array([-3., -1.,  2.,  6.]) array([3., 5., 6.]) array([11., 14., 15.])]
[3 2 2]


Based on these bin intervals, X is transformed as follows:

In [56]:
est.transform(X)

array([[0., 1., 1.],
       [1., 1., 1.],
       [2., 0., 0.]])

The resulting dataset contains **ordinal attributes** which can be further used in a `Pipeline`.

Discretization is similar to constructing histograms for continuous data. However, histograms focus on counting features which fall into particular bins, whereas discretization focuses on assigning feature values to these bins.

KBinsDiscretizer implements different binning strategies, which can be selected with the strategy parameter. 
* The `'uniform'` strategy uses constant-width bins. 
* The `'quantile'` strategy uses the quantiles values to have equally populated bins in each feature. 
* The `'kmeans'` strategy defines bins based on a k-means clustering procedure performed on each feature independently.

Be aware that one can specify custom bins by passing a callable defining the discretization strategy to `FunctionTransformer`. For instance, we can use the Pandas function `pandas.cut`:

In [67]:
import pandas as pd
import numpy as np

bins = [0, 1, 13, 20, 60, np.inf]
labels = ['infant', 'kid', 'teen', 'adult', 'senior citizen']

transformer = preprocessing.FunctionTransformer(  # Using FunctionTransformer...
    pd.cut, 
    kw_args={
        'bins': bins, 
        'labels': labels, 
        'retbins': False
    }
)

X = np.array([0.2, 2, 15, 25, 97])
transformer.fit_transform(X)

['infant', 'kid', 'teen', 'adult', 'senior citizen']
Categories (5, object): ['infant' < 'kid' < 'teen' < 'adult' < 'senior citizen']

**📚 Examples:**
* [Using KBinsDiscretizer to discretize continuous features](https://scikit-learn.org/stable/auto_examples/preprocessing/plot_discretization.html#sphx-glr-auto-examples-preprocessing-plot-discretization-py)
* [Feature discretization](https://scikit-learn.org/stable/auto_examples/preprocessing/plot_discretization_classification.html#sphx-glr-auto-examples-preprocessing-plot-discretization-classification-py)
* [Demonstrating the different strategies of KBinsDiscretizer](https://scikit-learn.org/stable/auto_examples/preprocessing/plot_discretization_strategies.html#sphx-glr-auto-examples-preprocessing-plot-discretization-strategies-py)

### Feature binarization

> Feature **binarization** is the process of **thresholding numerical features to get boolean values**. 

This can be useful for downstream probabilistic estimators that make assumption that the input data is distributed according to a multi-variate Bernoulli distribution. For instance, this is the case for the `BernoulliRBM`.

It is also common among the text processing community to use binary feature values (probably to simplify the probabilistic reasoning) even if normalized counts (a.k.a. term frequencies) or TF-IDF valued features often perform slightly better in practice.

As for the `Normalizer`, the utility class `Binarizer` is meant to be used in the early stages of `Pipeline`. 

⚠️ The `fit` method does nothing as each sample is treated independently of others:

In [68]:
X = [
    [ 1., -1.,  2.],
    [ 2.,  0.,  0.],
    [ 0.,  1., -1.]
]

binarizer = preprocessing.Binarizer().fit(X)  # fit does nothing
binarizer

Binarizer()

In [69]:
binarizer.transform(X)

array([[1., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.]])

It is possible to **adjust the threshold** of the binarizer:

In [70]:
binarizer = preprocessing.Binarizer(threshold=1.1)
binarizer.transform(X)

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 0.]])

As for the `Normalizer` class, the preprocessing module provides a companion function `binarize` to be used when the transformer API is not necessary.

Note that the `Binarizer` is similar to the `KBinsDiscretizer` when `k = 2`, and when the bin edge is at the value threshold.

---

**Sparse input:**

`binarize` and `Binarizer` **accept both** dense array-like and sparse matrices from `scipy.sparse` as input.

For sparse input the data is converted to the `Compressed Sparse Rows` representation (see `scipy.sparse.csr_matrix`). To avoid unnecessary memory copies, it is recommended to choose the CSR representation upstream.

---

## [Imputation of missing values](https://scikit-learn.org/stable/modules/preprocessing.html#imputation-of-missing-values)

* ➯ 📚 Tools for imputing missing values are discussed at [Imputation of missing values](https://scikit-learn.org/stable/modules/impute.html#impute).

## [Generating polynomial features](https://scikit-learn.org/stable/modules/preprocessing.html#generating-polynomial-features)

Often it’s useful to add complexity to the model by considering nonlinear features of the input data. A simple and common method to use is polynomial features, which can get features’ high-order and interaction terms. It is implemented in `PolynomialFeatures`:

In [71]:
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
X = np.arange(6).reshape(3, 2)
X

array([[0, 1],
       [2, 3],
       [4, 5]])

In [72]:
poly = PolynomialFeatures(2)
poly.fit_transform(X)

array([[ 1.,  0.,  1.,  0.,  0.,  1.],
       [ 1.,  2.,  3.,  4.,  6.,  9.],
       [ 1.,  4.,  5., 16., 20., 25.]])

The features of `X` have been transformed from $(X_1, X_2)$ to $(1, X_1, X_2, X_1^2, X_1 X_2, X_2^2)$.

In some cases, only interaction terms among features are required, and it can be gotten with the setting `interaction_only=True`:

In [73]:
X = np.arange(9).reshape(3, 3)
X

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [74]:
poly = PolynomialFeatures(
    degree=3, 
    interaction_only=True  # <-- Note this.
)
poly.fit_transform(X)

array([[  1.,   0.,   1.,   2.,   0.,   0.,   2.,   0.],
       [  1.,   3.,   4.,   5.,  12.,  15.,  20.,  60.],
       [  1.,   6.,   7.,   8.,  42.,  48.,  56., 336.]])

The features of `X` have been transformed from $(X_1, X_2, X_3)$ to $(1, X_1, X_2, X_3, X_1 X_2, X_1 X_3, X_2 X_3, X_1 X_2 X_3)$.

Note that polynomial features are used *implicitly* in kernel methods (e.g., `SVC`, `KernelPCA`) when using *polynomial Kernel functions*.

See [Polynomial interpolation](https://scikit-learn.org/stable/auto_examples/linear_model/plot_polynomial_interpolation.html#sphx-glr-auto-examples-linear-model-plot-polynomial-interpolation-py) for Ridge regression using created polynomial features.

## [💡 Custom transformers](https://scikit-learn.org/stable/modules/preprocessing.html#custom-transformers)

Often, you will want to convert an existing Python function into a transformer to assist in data cleaning or processing. You can implement a transformer from an arbitrary function with FunctionTransformer. 

For example, to build a transformer that applies a log transformation in a pipeline, do:

In [75]:
import numpy as np
from sklearn.preprocessing import FunctionTransformer

transformer = FunctionTransformer(
    np.log1p,  # <-- The function to apply is set here.
    validate=True
)

X = np.array([[0, 1], [2, 3]])
print(X)

X = transformer.transform(X)
print(X)

[[0 1]
 [2 3]]
[[0.         0.69314718]
 [1.09861229 1.38629436]]


You can ensure that `func` and `inverse_func` are the inverse of each other by setting `check_inverse=True` and calling `fit` before transform. 

Please note that a **warning** is raised and can be turned into an **error** with a `filterwarnings`:

```python
import warnings
warnings.filterwarnings(
    "error", 
    message=".*check_inverse*.",
    category=UserWarning, 
    append=False
)
```